In [2]:
import pennylane as qml 
from pennylane import numpy as np 
import numpy as scinp 
import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = [10, 10]
import torch
from torch.nn.init import xavier_uniform_
from torch import nn 

In [3]:
latent_dimensionality=2

In [4]:
dev=qml.device("qulacs.simulator",wires=latent_dimensionality,gpu=True)

In [5]:
def state_preparation(latent_variables):
    for i in range(len(latent_variables)):
        qml.RX(latent_variables[i],wires=i)

In [6]:
def layer(W):
    num_wires=len(W)
    for i in range(num_wires):
        random_rot=np.random.choice([qml.RX,qml.RY,qml.RZ])
        random_rot(W[i],wires=i)
        
    for i in range(num_wires-1):
        qml.CNOT(wires=[i,i+1])
        

In [7]:
@qml.qnode(dev,interface='torch')
def variational_circuit(latent_variables,weights):
    # The number of wires is exactly equal to 
    # the dimensionality of the latent variables.
    
    state_preparation(latent_variables)

    for W in weights:
        layer(W)
    
    return [qml.expval(qml.PauliZ(i)) for i in range(len(latent_variables))]

# We test our variational circuit by generating random variables and a glorot uniform distributed weights array, and drawing the circuit and evaluating it. 

In [8]:
latent_variables=scinp.random.uniform(-np.pi,np.pi,size=(latent_dimensionality,))
latent_variables

array([-1.1431329 ,  2.98086657])

In [9]:
num_layers=2

In [10]:
W=torch.empty(num_layers,latent_dimensionality)
W=xavier_uniform_(W)
W

tensor([[ 0.2549,  0.2910],
        [-0.4595,  0.2508]])

In [11]:
drawer=qml.draw(variational_circuit)

In [12]:
# We draw the circuit. 
variational_circuit(latent_variables,W)
print(variational_circuit.draw())

 0: ──RX(-1.14)──RY(0.255)──╭C──RY(-0.46)──╭C──┤ ⟨Z⟩ 
 1: ──RX(2.98)───RX(0.291)──╰X──RX(0.251)──╰X──┤ ⟨Z⟩ 



In [13]:
variational_circuit(latent_variables,W)

tensor([ 0.3466, -0.7446], dtype=torch.float64)

# We have shown that the code for the variational circuit is working correctly. We can now proceed with the rest of the algorithm. 

In [14]:
# We define the Torch generator now.

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

Using cuda:0 device


In [16]:
class QuantumGenerator(nn.Module):
    
    def __init__(self,variational_quantum_circuit,latent_dim,num_layers,upscaling_dimension,device):
        super(QuantumGenerator,self).__init__()
        
        self.device=device
        self.latent_dim=latent_dim
        # We initalize and store the quantum classifier's weights
        W=torch.Tensor(num_layers,latent_dim).uniform_(-np.pi,np.pi).to(self.device)
        
        # We specify that the quantum classifier weights parameters of the 
        # hybrid quantum classical generator, and thus should be differentiated.
        
        self.quantum_weights=nn.Parameter(W)
        
        # We store the quantum classifier
        self.vqc=variational_quantum_circuit
                
        # We define the upscaling layer, and we initialize it using the 
        # glorot uniform weight initialization
        self.upscaling_layer=nn.Linear(latent_dim,upscaling_dimension)
        xavier_uniform_(self.upscaling_layer.weight)
        
    def forward(self):
        # We define the latent variables, and pass them through a quantum generator.
        latent_variables=torch.Tensor(self.latent_dim).uniform_(-np.pi,np.pi).to(self.device)
        quantum_out=torch.Tensor(0,self.latent_dim).to(self.device)
        exp_val=self.vqc(latent_variables,self.quantum_weights).float().unsqueeze(0).to(self.device)
        quantum_out=torch.cat((quantum_out,exp_val))
        generated_sample=self.upscaling_layer(quantum_out)
        return generated_sample

In [17]:
quantum_gen=QuantumGenerator(variational_circuit,latent_dimensionality,num_layers,4,device=device)
quantum_gen=quantum_gen.to(device)

In [18]:
quantum_gen()

tensor([[ 0.7620, -1.0604, -0.4739,  0.1628]], device='cuda:0',
       grad_fn=<AddmmBackward>)

# We now try backpropagation with respect to some toy function to make sure that we know how to differentiate properly.

In [19]:
y_true=torch.Tensor(1,4).uniform_(-5,5).to(device)
y_true

tensor([[-2.1523, -2.5327, -2.5577,  2.1976]], device='cuda:0')

In [20]:
y_pred=quantum_gen()

In [21]:
loss=nn.functional.mse_loss(input=y_pred,target=y_true)

# We backpropagate through the loss, and we print the gradients 

In [22]:
loss.backward()

In [23]:
quantum_gen.quantum_weights.grad

tensor([[-0.0298, -0.3020],
        [-0.2427,  0.3001]], device='cuda:0')

In [24]:
quantum_gen.upscaling_layer.weight.grad

tensor([[ 0.3751,  0.0772],
        [ 0.3654,  0.0752],
        [ 0.3363,  0.0692],
        [-0.4119, -0.0847]], device='cuda:0')

# We print the parameters of our hybrid quantum classical generator.

In [25]:
quantum_gen.quantum_weights

Parameter containing:
tensor([[ 2.1662, -1.1994],
        [ 1.8532,  0.5611]], device='cuda:0', requires_grad=True)

In [26]:
quantum_gen.upscaling_layer.weight

Parameter containing:
tensor([[ 0.0050,  0.8195],
        [ 0.2451, -0.7825],
        [-0.2430, -0.0798],
        [-0.3815,  0.3488]], device='cuda:0', requires_grad=True)

In [27]:
quantum_gen.upscaling_layer.bias

Parameter containing:
tensor([ 0.0578, -0.3524, -0.4396, -0.1911], device='cuda:0',
       requires_grad=True)

In [28]:
list(quantum_gen.parameters())

[Parameter containing:
 tensor([[ 2.1662, -1.1994],
         [ 1.8532,  0.5611]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[ 0.0050,  0.8195],
         [ 0.2451, -0.7825],
         [-0.2430, -0.0798],
         [-0.3815,  0.3488]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([ 0.0578, -0.3524, -0.4396, -0.1911], device='cuda:0',
        requires_grad=True)]

# We saw in the previous cells that the parameters obtained by calling quantum_gen.parameters() are exactly those we want to optimize, including those of the quantum circuit. We can now feel comfortable using the torch ADAM optimizer to step.

In [29]:
optimizer=torch.optim.Adam(quantum_gen.parameters())

In [30]:
optimizer.step()

# We print the new parameters.

In [31]:
list(quantum_gen.parameters())

[Parameter containing:
 tensor([[ 2.1672, -1.1984],
         [ 1.8542,  0.5601]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[ 0.0040,  0.8185],
         [ 0.2441, -0.7835],
         [-0.2440, -0.0808],
         [-0.3805,  0.3498]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([ 0.0568, -0.3534, -0.4406, -0.1901], device='cuda:0',
        requires_grad=True)]

In [39]:
for a in quantum_gen.parameters():
    print(a.shape)

torch.Size([2, 2])
torch.Size([4, 2])
torch.Size([4])


In [ ]:
quant